In [ ]:
__autor__ = "Felipe Alejandro Pizarro Márquez"
__credits__ = ["Felipe Alejandro Pizarro Márquez",]
__email__ = "fpizarro92@live.cl"
__status__ = "Development"

## Import image set and call external sofware for photogrammetry

This process take a set of images and create a fully-textured 3D mesh using photogrammetry. To accomplish this, first it generate a dense cloud point with the image set using [openMVG](https://github.com/openMVG/openMVG) . Then, it uses [openMVS](https://github.com/cdcseacave/openMVS) to generate a fully-textured 3D mesh. 

In [ ]:
import os
import subprocess
import shutil
import PIL.Image
from tkinter import *
from tkinter import filedialog

In [ ]:
def create_3dmodel(project_name = "default_project",**kwargs):
    """     
    Arguments:
        project_name: The name of the project
    
    Optional Arguments:
        input_path:  It's the folder where the original images are stored.
        output_path: It's the absolute path of the output folder.
    """ 
    s3path = kwargs.pop('s3path',os.path.join(os.path.expanduser('~'),'super-scanner-software-s3'))
    input_path = kwargs.pop('input_path', False)
    output_path = kwargs.pop('output_path', False)
    scale = kwargs.pop('scale', 100)
    sample_path = os.path.join(s3path,"img","photog")
    out_path = os.path.join(os.path.expanduser('~'),'s3-out')
    
    if (input_path is False):
        input_path = select_folder(initialdir=sample_path,title = "Select input image data folder.")
        
    assert len(input_path) is not 0,"no input folder selected."
    
    if (output_path is False):    
        mkdir_ine(out_path)
        output_path = select_folder(initialdir=out_path,title = "Select output folder.")
    
    assert len(output_path) is not 0,"no output folder selected." 
    
    project_path = os.path.join(output_path,project_name)
    temp_path = os.path.join(project_path,"res-images")
    
    print("Input image path: %s" %input_path)
    print("Project path: %s"  %project_path)
    
    if os.path.isdir(project_path):
        print("The project with the name %s already exist" % project_name)
        return project_path
    # copy and scale the images to the project path
    if scale == 100:
        man_input_img(input_path,temp_path)
    else:
        man_input_img(input_path,temp_path, scale = scale/100)
    mvgmvspipeline = os.path.join(s3path, "scripts", "mvgmvspipeline.py")
    #call the python script pipeline to use openmvg and openmvs with custom parameters 
    subprocess.call(['python3',mvgmvspipeline,temp_path,project_path,"--1","m","AKAZE_FLOAT","p","HIGH","--2","f","1","r","0.7"])     
    return project_path   

In [ ]:
def get_3dmodel(**kwargs):
    """
    Optional Arguments:
        editor:       It's the object's editor. Currently only MESHLAB.
        project_path: It's project path.  
    """    
    out_path = os.path.join(os.path.expanduser('~'),'s3-out')    
    project_path = kwargs.pop('project_path', False)
    if (project_path is False):   
        mkdir_ine(out_path)
        project_path = select_folder(initialdir=out_path,title = "Select project folder.")
    
    assert len(project_path) is not 0,"no project folder selected"    

    obj = os.path.join(project_path, "mvs","scene_dense_mesh_refine_texture.ply")
    print("Project path: %s"  % project_path)
    assert os.path.isfile(obj) is True, "incorrect project folder"    
    
    #Check if the selected editor exist
    editor_switcher = {"MESHLAB": "meshlab"}
    editor = editor_switcher.get(kwargs.pop('editor', 'MESHLAB'),False)        
    assert editor != False,"incorrect editor"
        
    if editor is "meshlab":
    #Use meshlab to edit the mesh        
        subprocess.Popen([editor,obj])

In [ ]:
def import_3dmodel(**kwargs):
    """
    Import any standford (.ply) mesh from any path.

    Optional Arguments:
        object_path:  The mesh to import.
        editor:       It's the object's editor. Currently only MESHLAB.
    """  
    out_path = os.path.join(os.path.expanduser('~'),'s3-out')    
    object_path = kwargs.pop('object_path', False)
    
    if (object_path is False):
        mkdir_ine(out_path) 
        object_path = select_object(initialdir=out_path,title = "Select Standford file.",filetypes = [('Standford', '.ply')])
    assert len(object_path) is not 0,"no standford object selected"  
    
    #Check if the selected editor exist
    editor_switcher = {"MESHLAB": "meshlab"}
    editor = editor_switcher.get(kwargs.pop('editor', 'MESHLAB'),False)        
    assert editor != False,"incorrect editor"
                
    if editor is "meshlab":
    #Use meshlab to edit the mesh        
        subprocess.Popen([editor,object_path])

In [ ]:
def mkdir_ine(dirname):
    """Create the folder if not presents"""
    if not os.path.exists(dirname):
        os.makedirs(dirname)

In [ ]:
def select_folder(**kwargs):
    """
    Return a folder path selected from tkinter askdirectory
    Optional Arguments:
        initial_dir:  It's where to start looking for a folder
        title:        The title of the windows screen
    """
    initialdir = kwargs.pop('initialdir', os.path.expanduser('~'))
    title = kwargs.pop('title', 'Please choose a directory, then press OK.')
    root = Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    return filedialog.askdirectory(initialdir=initialdir, parent=root,title = title,mustexist = True)

In [ ]:
def select_object(**kwargs):
    """
    Return a object_path selected from tkinter askopenfilename
    Optional Arguments:
        initial_dir:  It's where to start looking for an object
        title:        The title of the windows screen
        filetypes:    The filetype of the object
    """
    initialdir = kwargs.pop('initialdir', os.path.expanduser('~'))
    title = kwargs.pop('title', 'Please choose a file, then press OK.')
    filetypes = kwargs.pop('filetypes', [('all files', '.*')])
    root = Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    return filedialog.askopenfilename(initialdir=initialdir,title = title,filetypes = filetypes)

In [ ]:
def man_input_img(input_path,output_path,**kwargs):
    """
    Manages the input image path
    """
    scale = kwargs.pop('scale', False)
    exts = ('.jpg', '.JPG')
    os.makedirs(output_path) 
    for file in os.listdir(input_path):
        if os.path.isfile(os.path.join(input_path, file)):
            if any(file.lower().endswith(ext) for ext in exts):
                shutil.copy2(os.path.join(input_path, file),os.path.join(output_path,file))
                if scale != False:
                    resize_image(os.path.join(output_path,file),scale)

In [ ]:
def resize_image(img_path, s_factor):
    """
    Scales the images size with a s_factor
    """    
    image = open(img_path, "rb")
    img = PIL.Image.open(image)
    exif = img.info['exif']
    img = img.resize([int(s_factor * s) for s in img.size], PIL.Image.ANTIALIAS)
    img.save(img_path, exif = exif)
    image.close()